In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib

In [21]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer

In [22]:
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.model_selection import cross_val_predict, cross_val_score
from sklearn.svm import SVC
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import make_scorer
from scipy.stats import randint, uniform, loguniform

In [23]:
from lifelines import KaplanMeierFitter
from sksurv.ensemble import RandomSurvivalForest
from sksurv.ensemble import GradientBoostingSurvivalAnalysis
from sksurv.tree import SurvivalTree
from sksurv.meta import Stacking
from sksurv.linear_model import CoxPHSurvivalAnalysis

In [24]:
# Import dataset
df = pd.read_csv('dataset_da_2017.csv', encoding='utf-8')
print(df.head())

   Patient ID  Year of diagnosis     Sex  Age   Race Median household income  \
0        3614               2017  Female   74  White               ≥ $75,000   
1        4135               2017  Female   78  White               ≥ $75,000   
2        4432               2017  Female   69  White               ≥ $75,000   
3        7922               2017  Female   79  White               ≥ $75,000   
4       66852               2017  Female   81  White               ≥ $75,000   

      Marital status at diagnosis    Grade Combined Summary Stage Stage Group  \
0  Married (including common law)      III                Distant          IV   
1                         Widowed  Unknown                Distant          IV   
2                        Divorced  Unknown                Distant          IV   
3                        Divorced       II               Regional          II   
4  Married (including common law)      III               Regional         III   

   ... Surgery Radiation Chemoth

In [25]:
# Drop some unuseful columns
columns_to_drop = ['Patient ID', 'Year of diagnosis', 'Survival months flag']
df = df.drop(columns=columns_to_drop)
print(df.head())
print(df.shape)
# Replace the unknown values with NaNs
df.replace("Unknown", np.nan, inplace=True)
print(df.head())

      Sex  Age   Race Median household income     Marital status at diagnosis  \
0  Female   74  White               ≥ $75,000  Married (including common law)   
1  Female   78  White               ≥ $75,000                         Widowed   
2  Female   69  White               ≥ $75,000                        Divorced   
3  Female   79  White               ≥ $75,000                        Divorced   
4  Female   81  White               ≥ $75,000  Married (including common law)   

     Grade Combined Summary Stage Stage Group Tumor Size Surgery Radiation  \
0      III                Distant          IV         50      No   Unknown   
1  Unknown                Distant          IV         56      No   Unknown   
2  Unknown                Distant          IV         66      No   Unknown   
3       II               Regional          II         23     Yes   Unknown   
4      III               Regional         III         68      No       Yes   

  Chemotherapy Mets at DX-bone Mets at DX-br

In [26]:
# Define X and y
X = df.drop(['Survival months', 'Vital status'], axis=1)
y_surv = df['Survival months']
y_vital = df['Vital status']
# Split data for 70% train and 30% test
x_train, x_test, y_surv_train, y_surv_test, y_vital_train, y_vital_test = train_test_split(
    X, y_surv, y_vital, test_size=0.3, random_state=42, stratify=y_vital)

In [27]:
# Create mapping dictionaries
income_mapping = {'< $35,000': 0, '$35,000 - $39,999': 1, '$40,000 - $44,999': 2, '$45,000 - $49,999': 3, '$50,000 - $54,999': 4,
                  '$55,000 - $59,999': 5, '$60,000 - $64,999': 6, '$65,000 - $69,999': 7, '$70,000 - $74,999': 8, '≥ $75,000': 9}
grade_mapping = {'I': 0, 'II': 1, 'III': 2, 'IV': 3}
stage_mapping = {'I': 0, 'II': 1, 'III': 2, 'IV': 3}
summary_stage_mapping = {'Localized': 0, 'Regional': 1, 'Distant': 2}

# Encode the ordinal variables
x_train['Median household income'] = x_train['Median household income'].map(income_mapping)
x_test['Median household income'] = x_test['Median household income'].map(income_mapping)

x_train['Grade'] = x_train['Grade'].map(grade_mapping)
x_test['Grade'] = x_test['Grade'].map(grade_mapping)

x_train['Stage Group'] = x_train['Stage Group'].map(stage_mapping)
x_test['Stage Group'] = x_test['Stage Group'].map(stage_mapping)

x_train['Combined Summary Stage'] = x_train['Combined Summary Stage'].map(summary_stage_mapping)
x_test['Combined Summary Stage'] = x_test['Combined Summary Stage'].map(summary_stage_mapping)

In [28]:
# Nominal variables
nominal_vars = ["Sex", "Race", "Marital status at diagnosis", "Surgery", "Radiation",
                "Chemotherapy", "Mets at DX-bone", "Mets at DX-brain", "Mets at DX-lung",
                "Mets at DX-liver"]

# Perform One-Hot encoding on nominal variables
x_train = pd.get_dummies(x_train, columns=nominal_vars)
x_test = pd.get_dummies(x_test, columns=nominal_vars)

In [29]:
x_train["Tumor Size"] = pd.to_numeric(x_train["Tumor Size"], errors="coerce")
x_test["Tumor Size"] = pd.to_numeric(x_test["Tumor Size"], errors="coerce")
print(x_train.dtypes)
print(x_train.head())

Age                                                             int64
Median household income                                         int64
Grade                                                         float64
Combined Summary Stage                                        float64
Stage Group                                                     int64
Tumor Size                                                    float64
Sex_Female                                                      uint8
Sex_Male                                                        uint8
Race_American Indian/Alaska Native                              uint8
Race_Asian or Pacific Islander                                  uint8
Race_Black                                                      uint8
Race_White                                                      uint8
Marital status at diagnosis_Divorced                            uint8
Marital status at diagnosis_Married (including common law)      uint8
Marital status at di

In [30]:
# Save the column names and indexes before imputation
columns = x_train.columns.tolist()
train_index = x_train.index
test_index = x_test.index


In [31]:
%%time
# Initialize the KNNImputer with 5 neighbors
imputer = KNNImputer(n_neighbors=5)  
# Fit the imputer on the training data and transform it
x_train_imputed = imputer.fit_transform(x_train)
# Transform the test data
x_test_imputed = imputer.transform(x_test)

CPU times: total: 38.7 s
Wall time: 37.9 s


In [32]:
# Convert back to pandas dataframe
x_train_df = pd.DataFrame(x_train_imputed, columns=columns, index=train_index)
x_test_df = pd.DataFrame(x_test_imputed, columns=columns, index=test_index)
print(x_train_df.columns)
print(x_train_df.index)
# Convert into integer
cols_to_convert = ['Age', 'Median household income', 'Grade', 'Stage Group', 'Combined Summary Stage']
for col in cols_to_convert:
    x_train_df[col] = x_train_df[col].astype(int)
    x_test_df[col] = x_test_df[col].astype(int)

Index(['Age', 'Median household income', 'Grade', 'Combined Summary Stage',
       'Stage Group', 'Tumor Size', 'Sex_Female', 'Sex_Male',
       'Race_American Indian/Alaska Native', 'Race_Asian or Pacific Islander',
       'Race_Black', 'Race_White', 'Marital status at diagnosis_Divorced',
       'Marital status at diagnosis_Married (including common law)',
       'Marital status at diagnosis_Separated',
       'Marital status at diagnosis_Single (never married)',
       'Marital status at diagnosis_Unmarried or Domestic Partner',
       'Marital status at diagnosis_Widowed', 'Surgery_No', 'Surgery_Yes',
       'Radiation_No', 'Radiation_Yes', 'Chemotherapy_No', 'Chemotherapy_Yes',
       'Mets at DX-bone_No', 'Mets at DX-bone_Yes', 'Mets at DX-brain_No',
       'Mets at DX-brain_Yes', 'Mets at DX-lung_No', 'Mets at DX-lung_Yes',
       'Mets at DX-liver_No', 'Mets at DX-liver_Yes'],
      dtype='object')
Int64Index([ 6636, 24979, 12378, 15263, 21547, 17873, 28370,  2227, 12896,
     

In [33]:
#  Normalisation using StandardScaler
scaler = StandardScaler()
columns_to_scale = ['Age', 'Tumor Size']
x_train_df[columns_to_scale] = scaler.fit_transform(x_train_df[columns_to_scale])
x_test_df[columns_to_scale] = scaler.transform(x_test_df[columns_to_scale])

In [34]:
# Reset indices to ensure consistency
x_train_df = x_train_df.reset_index(drop=True)
x_test_df = x_test_df.reset_index(drop=True)
y_surv_train = y_surv_train.reset_index(drop=True)
y_surv_test = y_surv_test.reset_index(drop=True)
y_vital_train = y_vital_train.reset_index(drop=True)
y_vital_test = y_vital_test.reset_index(drop=True)

# Combine features and target variables
train_data_new = pd.concat([x_train_df, y_surv_train, y_vital_train], axis=1)
test_data_new = pd.concat([x_test_df, y_surv_test, y_vital_test], axis=1)

# Map 'Alive' to 0 and 'Dead' to 1
train_data_new['Vital status'] = train_data_new['Vital status'].map({'Alive': 0, 'Dead': 1})
test_data_new['Vital status'] = test_data_new['Vital status'].map({'Alive': 0, 'Dead': 1})

# Function to convert DataFrame to structured array for scikit-survival
def df_to_structured(df, event_col, time_col):
    return np.array([(e, t) for e, t in zip(df[event_col], df[time_col])], dtype=[('event', bool), ('time', np.float64)])

# Convert training and testing data
y_train_structured = df_to_structured(train_data_new, 'Vital status', 'Survival months')
y_test_structured = df_to_structured(test_data_new, 'Vital status', 'Survival months')

In [35]:
%%time
from lifelines import CoxPHFitter
from lifelines.utils import median_survival_times
from lifelines.utils import concordance_index

# Train the Cox Proportional Hazards model with L2 regularization
coxph = CoxPHFitter(penalizer=0.3593813663804626)
coxph.fit(train_data_new, duration_col='Survival months', event_col='Vital status')

# Predict the mean survival time for each individual in the test set
predicted_means  = coxph.predict_expectation(test_data_new.drop(['Vital status', 'Survival months'], axis=1))

# Convert the predictions to a DataFrame for further processing
predicted_means = pd.DataFrame(predicted_means, columns=['Predicted Survival Time (Mean)'])
print(f"Predicted survival times (Mean): {predicted_means}")

Predicted survival times (Mean):        Predicted Survival Time (Mean)
0                           15.569164
1                           22.890884
2                           30.832365
3                           14.951649
4                           24.585819
...                               ...
10433                       21.943356
10434                       30.353859
10435                        7.560636
10436                       18.577583
10437                       10.467992

[10438 rows x 1 columns]
CPU times: total: 62.5 ms
Wall time: 584 ms


In [36]:
# save the model to disk
joblib.dump(coxph, "cox_model.sav")

['cox_model.sav']